In [395]:
import midi
import numpy as np

In [489]:
class MidiContainer:
    '''
    The MidiContainer class contains a vector representation of 
    the midi track to be fed to our neural net. It is a 67 x n matrix,
    contents specified in the README. 
    '''
    def __init__(self):
        self.data = np.array([], dtype=np.int64).reshape(66,0) 
        self.curr_bpm = 120
        self.curr_instrument = 0
        self.abs_time = 0
        self.active_pitches = {} #[pitch] = (velocity, time note was turned on)
        
    def note_to_vector(self, pitch, time):
        velocity = self.active_pitches[pitch][0]
        note_start = self.active_pitches[pitch][1]
        return np.vstack((self.curr_instrument, pitch, velocity, time + self.abs_time - note_start))
    
    def add_data(self, note_vector, note_start):
        event_column = np.vstack((note_start, self.curr_bpm, note_vector, np.ones((60,1))*-1))
        self.data = np.hstack((self.data, event_column))
    
    def combine_tracks(self):
        self.data = self.data.T[self.data.T[:, 0].argsort()].T #maybe we dont need so many transposes
        prev_duration = -1
        edit_index = 0 
        i = 0
        delete_this = []
        old_inst = self.data[2,0]
        for col in self.data.T:
            curr_duration = col[0]
            if curr_duration == prev_duration:
                for ni in xrange(2,67,4):
                    note_vec = col[ni:ni+4]
                    curr_inst = note_vec[0] if note_vec.any() else -1
                    if np.sum(note_vec) > 0 and curr_inst != old_inst:
                        i_rep = np.where(self.data[:,edit_index] == -1)[0][0]
                        self.data[i_rep:i_rep+4, edit_index] = note_vec
                        delete_this.append(i)
            else:
                old_inst = col[2]
                edit_index = i
            i += 1 
            prev_duration = curr_duration
            
        mask = np.ones(self.data.shape, np.bool)
        mask[:,delete_this] = 0
        self.data = self.data[mask].reshape(66,-1)
       
                

In [466]:
def midi_to_vector(fname):
    '''
    Given a filename of a midi to read, returns a MidiContainer object of its vector representation.
    '''
    pattern = midi.read_midifile(fname)
    midi_vector = MidiContainer()
    for track in pattern:
        midi_vector.abs_time = 0
        for event in track:
            if isinstance(event, midi.SetTempoEvent): 
                midi_vector.curr_bpm = event.get_bpm()
            if isinstance(event, midi.EndOfTrackEvent): #first metadata track
                continue 
            if isinstance(event, midi.ProgramChangeEvent):
                midi_vector.curr_instrument = event.get_value()
            if isinstance(event, midi.NoteOnEvent):
                midi_vector.abs_time += event.tick
                midi_vector.active_pitches[event.get_pitch()] = (event.get_velocity(), midi_vector.abs_time)
            if isinstance(event, midi.NoteOffEvent):
                pitch = event.get_pitch()
                time = event.tick
                if pitch in midi_vector.active_pitches:
                    note_vec = midi_vector.note_to_vector(pitch, time)
                    midi_vector.add_data(note_vec, midi_vector.active_pitches[pitch][1]) #ugh bad data abstraction
                    midi_vector.active_pitches.pop(pitch, None)
                    midi_vector.abs_time += event.tick
                
    midi_vector.combine_tracks()        
    return midi_vector

In [481]:
def vector_to_midi(vector):
    '''Given a np array (vector.data), returns a python midi pattern'''
    pattern = midi.Pattern(resolution=960)
    #first, separate tracks
    tracklist = []
    for i in xrange(2,vector.shape[0],4):
        track = vector[i:i+4,:]
        if np.sum(track) > 0:
            tracklist.append(np.vstack((vector[0:2], track)))
   
    for track in tracklist:
        miditrack = midi.Track()
        pattern.append(miditrack)
        bpm = track[1,0]
        tempoevent = midi.SetTempoEvent(tick=0, bpm=bpm)
        miditrack.append(tempoevent)
        instrument = int(track[2,0])
        fontevent = midi.ProgramChangeEvent(tick=0, value=instrument)
        miditrack.append(fontevent)
        if not np.array_equal(bpm * np.ones(track.shape[1]), track[1,:]):
            print 'Tempo changes. Code assumes it doesn\'t. Contact Jingyi.'
        
        event_tick = 0   
        track_duration = np.max(track[0,:] + track[-1,:])
        active_notes = {}
        start_times = track[0,:]
        for t in xrange(0, int(track_duration)+1):
            for pitch in active_notes:
                active_notes[pitch] -= 1
                if t % 960 == 0: print 'snapshot at ', t, active_notes
            
            negs = [k for k,v in active_notes.iteritems() if v < 0]
            for n in negs:
                print 'removing negative ', n
                active_notes.pop(n)
                
            while 0 in active_notes.values():
                pitches = [k for k,v in active_notes.iteritems() if v == 0]
                for pitch in pitches:
                    print 'note ', pitch, ' got to 0 '
                    print 'active notes are ', active_notes 
                    off = midi.NoteOffEvent(tick=t - event_tick, pitch=pitch)
                    print ' Added off event with t ', t- event_tick, 'for pitch ', pitch
                    miditrack.append(off)
                    active_notes.pop(pitch)
                    event_tick = t
                    #relatively subtract
                    for pitch in active_notes:
                        active_notes[pitch] -= event_tick
                    if active_notes:
                        if sum(active_notes.itervalues()) > 0:
                            print "SUBBING", 'active notes is ', active_notes
                            start_times -= event_tick
                            event_tick = 0
                    print 'Event tick is now ', event_tick, ' and start times ', start_times
                    print 'Active notes are ', active_notes
                    
            #run through track to add on/off events
            if t in start_times:
                ni = np.where(t == start_times)
                for n in ni[0]:
                    note = track[:, n]
                    start_time = int(note[0])
                    pitch = int(note[3])
                    velocity = int(note[4])
                    duration = int(note[5])
                    active_notes[pitch] = duration
                    on = midi.NoteOnEvent(tick = t - event_tick, velocity=velocity, pitch=pitch)
                    miditrack.append(on)
        
                    print 'Added on event: ', note, 'at time ', t - event_tick
                    event_tick = start_time
                    print ' event tick now ', event_tick
                    
        miditrack.append(midi.EndOfTrackEvent(tick=0))
    return pattern

In [492]:
m = midi_to_vector('multitrack.mid')
print 'm is!!!!!', m.data
p = vector_to_midi(m.data)
print 'p is !!!!!!!'
print p
#midi.write_midifile('testunit1.mid',p)

m is!!!!! [[    0.   960.  1920.  2880.]
 [  120.   120.   120.   120.]
 [   90.    90.    90.    90.]
 [   76.    76.    76.    76.]
 [   80.    80.    80.    80.]
 [  960.   960.   960.   960.]
 [   75.    75.    75.    75.]
 [   69.    69.    68.    69.]
 [   80.    80.    80.    80.]
 [  960.   960.   960.   960.]
 [   -1.    -1.    -1.    -1.]
 [   -1.    -1.    -1.    -1.]
 [   -1.    -1.    -1.    -1.]
 [   -1.    -1.    -1.    -1.]
 [   -1.    -1.    -1.    -1.]
 [   -1.    -1.    -1.    -1.]
 [   -1.    -1.    -1.    -1.]
 [   -1.    -1.    -1.    -1.]
 [   -1.    -1.    -1.    -1.]
 [   -1.    -1.    -1.    -1.]
 [   -1.    -1.    -1.    -1.]
 [   -1.    -1.    -1.    -1.]
 [   -1.    -1.    -1.    -1.]
 [   -1.    -1.    -1.    -1.]
 [   -1.    -1.    -1.    -1.]
 [   -1.    -1.    -1.    -1.]
 [   -1.    -1.    -1.    -1.]
 [   -1.    -1.    -1.    -1.]
 [   -1.    -1.    -1.    -1.]
 [   -1.    -1.    -1.    -1.]
 [   -1.    -1.    -1.    -1.]
 [   -1.    -1.    -1.    -1.

In [464]:
m = midi_to_vector('unit1.mid')
print 'm data is !!!!'
print m.data


note vec is  [[  0]
 [ 68]
 [ 80]
 [960]]
note vec is  [[  0]
 [ 67]
 [ 80]
 [960]]
note vec is  [[  0]
 [ 66]
 [ 80]
 [960]]
note vec is  [[  0]
 [ 65]
 [ 80]
 [960]]
note vec is  [[  0]
 [ 66]
 [ 80]
 [960]]
note vec is  [[  0]
 [ 67]
 [ 80]
 [960]]
note vec is  [[  0]
 [ 68]
 [ 80]
 [960]]
note vec is  [[  0]
 [ 69]
 [ 80]
 [960]]
m data is !!!!
[[  0.00000000e+00   9.60000000e+02   1.92000000e+03   2.88000000e+03
    3.84000000e+03   4.80000000e+03   5.76000000e+03   6.72000000e+03]
 [  2.00000000e+02   2.00000000e+02   2.00000000e+02   2.00000000e+02
    2.00000000e+02   2.00000000e+02   2.00000000e+02   2.00000000e+02]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  6.80000000e+01   6.70000000e+01   6.60000000e+01   6.50000000e+01
    6.60000000e+01   6.70000000e+01   6.80000000e+01   6.90000000e+01]
 [  8.00000000e+01   8.00000000e+01   8.00000000e+01   8.00000000e+01
    8.00000000e

In [444]:
midi.read_midifile('unit1.mid')

midi.Pattern(format=1, resolution=960, tracks=\
[midi.Track(\
  [midi.SetTempoEvent(tick=0, data=[4, 147, 224]),
   midi.KeySignatureEvent(tick=0, data=[0, 0]),
   midi.TimeSignatureEvent(tick=0, data=[4, 2, 24, 8]),
   midi.EndOfTrackEvent(tick=0, data=[])]),
 midi.Track(\
  [midi.ControlChangeEvent(tick=0, channel=0, data=[0, 0]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[32, 0]),
   midi.ProgramChangeEvent(tick=0, channel=0, data=[0]),
   midi.TrackNameEvent(tick=0, text='Untitled\x00', data=[85, 110, 116, 105, 116, 108, 101, 100, 0]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[7, 127]),
   midi.NoteOnEvent(tick=0, channel=0, data=[68, 80]),
   midi.NoteOffEvent(tick=960, channel=0, data=[68, 0]),
   midi.NoteOnEvent(tick=0, channel=0, data=[67, 80]),
   midi.NoteOffEvent(tick=960, channel=0, data=[67, 0]),
   midi.NoteOnEvent(tick=0, channel=0, data=[66, 80]),
   midi.NoteOffEvent(tick=960, channel=0, data=[66, 0]),
   midi.NoteOnEvent(tick=0, channel=0, data=[65

In [482]:
m2 = midi_to_vector('tempo.mid')
p2 = vector_to_midi(m2.data)
print 'p is !!!!!!!'
print p2

Added on event:  [   0.  200.    8.   71.   80.  960.] at time  0
 event tick now  0
Added on event:  [    0.   200.     8.    69.    80.  3840.] at time  0
 event tick now  0
Added on event:  [ 240.  200.    8.   73.   80.  240.] at time  240
 event tick now  240
note  73  got to 0 
active notes are  {73: 0, 69: 3360, 71: 480}
 Added off event with t  240 for pitch  73
SUBBING active notes is  {69: 2880, 71: 0}
Event tick is now  0  and start times  [ -480.  -480.  -240.   240.   480.   720.  1200.  1440.  1680.  2160.
  2400.  2640.  3120.]
Active notes are  {69: 2880, 71: 0}
note  71  got to 0 
active notes are  {69: 2880, 71: 0}
 Added off event with t  480 for pitch  71
SUBBING active notes is  {69: 2400}
Event tick is now  0  and start times  [ -960.  -960.  -720.  -240.     0.   240.   720.   960.  1200.  1680.
  1920.  2160.  2640.]
Active notes are  {69: 2400}
Added on event:  [ 720.  200.    8.   73.   80.  240.] at time  720
 event tick now  720
snapshot at  960 {73: 0, 69: 

In [480]:
np.set_printoptions(suppress=True)
m2.data.T

array([[    0.,   200.,     8.,    71.,    80.,   960.,    -1.,    -1.,
           -1.,    -1.,    -1.,    -1.,    -1.,    -1.,    -1.,    -1.,
           -1.,    -1.,    -1.,    -1.,    -1.,    -1.,    -1.,    -1.,
           -1.,    -1.,    -1.,    -1.,    -1.,    -1.,    -1.,    -1.,
           -1.,    -1.,    -1.,    -1.,    -1.,    -1.,    -1.,    -1.,
           -1.,    -1.,    -1.,    -1.,    -1.,    -1.,    -1.,    -1.,
           -1.,    -1.,    -1.,    -1.,    -1.,    -1.,    -1.,    -1.,
           -1.,    -1.,    -1.,    -1.,    -1.,    -1.,    -1.,    -1.,
           -1.,    -1.],
       [    0.,   200.,     8.,    69.,    80.,  3840.,    -1.,    -1.,
           -1.,    -1.,    -1.,    -1.,    -1.,    -1.,    -1.,    -1.,
           -1.,    -1.,    -1.,    -1.,    -1.,    -1.,    -1.,    -1.,
           -1.,    -1.,    -1.,    -1.,    -1.,    -1.,    -1.,    -1.,
           -1.,    -1.,    -1.,    -1.,    -1.,    -1.,    -1.,    -1.,
           -1.,    -1.,    -1.,    -1.,

In [473]:
midi.read_midifile('tempo.mid')

midi.Pattern(format=1, resolution=960, tracks=\
[midi.Track(\
  [midi.SetTempoEvent(tick=0, data=[4, 147, 224]),
   midi.KeySignatureEvent(tick=0, data=[0, 0]),
   midi.TimeSignatureEvent(tick=0, data=[4, 2, 24, 8]),
   midi.EndOfTrackEvent(tick=0, data=[])]),
 midi.Track(\
  [midi.ControlChangeEvent(tick=0, channel=0, data=[0, 0]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[32, 0]),
   midi.ProgramChangeEvent(tick=0, channel=0, data=[8]),
   midi.TrackNameEvent(tick=0, text='Untitled\x00', data=[85, 110, 116, 105, 116, 108, 101, 100, 0]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[7, 127]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[0, 0]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[32, 0]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[7, 127]),
   midi.NoteOnEvent(tick=0, channel=0, data=[69, 80]),
   midi.NoteOnEvent(tick=0, channel=0, data=[71, 80]),
   midi.NoteOnEvent(tick=240, channel=0, data=[73, 80]),
   midi.NoteOffEvent(tick=240, ch